In [3]:
import numpy as np
np.random.seed(20)
import pandas as pd

from tensorflow import set_random_seed
import t
from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn.model_selection import StratifiedKFold

'''Data loading & preprocessing
'''

X_train = pd.read_csv('data/train.csv')
X_test = pd.read_csv('data/test.csv')

X_train, y_train = X_train.iloc[:,2:], X_train.target
X_test, test_id = X_test.iloc[:,1:], X_test.id

#OHE / some feature engineering adapted from the1owl kernel at:
#https://www.kaggle.com/the1owl/forza-baseline/code

#excluded columns based on snowdog's old school nn kernel at:
#https://www.kaggle.com/snowdog/old-school-nnet

X_train['negative_one_vals'] = np.sum((X_train==-1).values, axis=1)
X_test['negative_one_vals'] = np.sum((X_test==-1).values, axis=1)

to_drop = ['ps_car_11_cat', 'ps_ind_14', 'ps_car_11', 'ps_car_14', 'ps_ind_06_bin', 
           'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 
           'ps_ind_13_bin']

cols_use = [c for c in X_train.columns if (not c.startswith('ps_calc_'))
             & (not c in to_drop)]
             
X_train = X_train[cols_use]
X_test = X_test[cols_use]

one_hot = {c: list(X_train[c].unique()) for c in X_train.columns}

#note that this encodes the negative_one_vals column as well
for c in one_hot:
    if len(one_hot[c])>2 and len(one_hot[c]) < 105:
        for val in one_hot[c]:
            newcol = c + '_oh_' + str(val)
            X_train[newcol] = (X_train[c].values == val).astype(np.int)
            X_test[newcol] = (X_test[c].values == val).astype(np.int)
        X_train.drop(labels=[c], axis=1, inplace=True)
        X_test.drop(labels=[c], axis=1, inplace=True)
            
X_train = X_train.replace(-1, np.NaN)  # Get rid of -1 while computing interaction col
X_test = X_test.replace(-1, np.NaN)

X_train['ps_car_13_x_ps_reg_03'] = X_train['ps_car_13'] * X_train['ps_reg_03']
X_test['ps_car_13_x_ps_reg_03'] = X_test['ps_car_13'] * X_test['ps_reg_03']

X_train = X_train.fillna(-1)
X_test = X_test.fillna(-1)

'''Gini scoring function
'''

#gini scoring function from kernel at: 
#https://www.kaggle.com/tezdhar/faster-gini-calculation
def ginic(actual, pred):
    n = len(actual)
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_c[-1] - (n + 1) / 2.0
    return giniSum / n
 
def gini_normalizedc(a, p):
    return ginic(a, p) / ginic(a, a)

'''5-fold neural network training 
'''

K = 5 #number of folds
runs_per_fold = 3 #bagging on each fold

cv_ginis = []
y_preds = np.zeros((np.shape(X_test)[0],K))

kfold = StratifiedKFold(n_splits = K, 
                            random_state = 100, 
                            shuffle = True)    


In [4]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name = 'weight')

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]),name='bias')


# ### Helper-function for creating a new Fully-Connected Layer
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True,  # Use Rectified Linear Unit (ReLU)?
                 use_sig=False):
    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)
    elif use_sig:
        layer = tf.nn.sigmoid(layer)
    return layer

n_nodes_hl1 = 57
n_nodes_hl2 = 35
n_nodes_hl3 = 15

beta = 0.002  # l2 regularization

keep_prob = tf.placeholder(tf.float32)

layer_h1 = new_fc_layer(input=x,
                         num_inputs=input_size,
                         num_outputs=n_nodes_hl1,
                         use_relu=False,
                         use_sig = True)
drop_h1 = tf.nn.dropout(layer_h1, 0.7 + keep_prob*0.2)

y_pred = new_fc_layer(input=drop_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=2,
                         use_relu=False,
                         use_sig = False)
y_pred_soft_max = tf.nn.sigmoid(y_pred)

var_s = tf.trainable_variables()
loss_l2 = tf.add_n([tf.nn.l2_loss(v) for v in var_s if 'bias' not in v.name])*0.001
loss = tf.square(y_pred_soft_max - y_true)/2
cost = tf.reduce_mean(loss+ loss_l2)
loss = tf.losses.sigmoid_cross_entropy(y_true,y_pred)

NameError: name 'tf' is not defined

In [ ]:
for i, (f_ind, outf_ind) in enumerate(kfold.split(X_train, y_train)):

    X_train_f, X_val_f = X_train.loc[f_ind].copy(), X_train.loc[outf_ind].copy()
    y_train_f, y_val_f = y_train[f_ind], y_train[outf_ind]
          
    #upsampling adapted from kernel: 
    #https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283
    pos = (pd.Series(y_train_f == 1))
    
    # Add positive examples
    X_train_f = pd.concat([X_train_f, X_train_f.loc[pos]], axis=0)
    y_train_f = pd.concat([y_train_f, y_train_f.loc[pos]], axis=0)
    
    # Shuffle data
    idx = np.arange(len(X_train_f))
    np.random.shuffle(idx)
    X_train_f = X_train_f.iloc[idx]
    y_train_f = y_train_f.iloc[idx]
    
    #track oof bagged prediction for cv scores
    val_preds = 0
    
    for j in range(runs_per_fold):
    
        NN=Sequential()
        NN.add(Dense(35,activation='relu',input_dim=np.shape(X_train_f)[1]))
        NN.add(Dropout(0.3))
        NN.add(Dense(1,activation='sigmoid'))
        
        NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        set_random_seed(1000*i+j)
            
        NN.fit(X_train_f.values, y_train_f.values, epochs=15, batch_size=2048, verbose=0)
         
        val_gini = gini_normalizedc(y_val_f.values, NN.predict(X_val_f.values)[:,0])   
        print ('\nFold %d Run %d Results *****' % (i, j))
        print ('Validation gini: %.5f\n' % (val_gini))
        
        val_preds += NN.predict(X_val_f.values)[:,0] / runs_per_fold
        y_preds[:,i] += NN.predict(X_test.values)[:,0] / runs_per_fold
        
    cv_ginis.append(val_gini)
    print ('\nFold %i prediction cv gini: %.5f\n' %(i,val_gini))
    
print('Mean out of fold gini: %.5f' % np.mean(cv_ginis))
y_pred_final = np.mean(y_preds, axis=1)

df_sub = pd.DataFrame({'id' : test_id, 
                       'target' : y_pred_final},
                       columns = ['id','target'])
df_sub.to_csv('NNShallow_5fold_3runs_sub.csv', index=False)